In [1]:
#Importing the libraries and getting the password to acces the database
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # to get the password without showing the input
password = getpass.getpass() # insert your password below after executing this cell

In [16]:
#Connecting to the database
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila' # database_location/database_name
engine = create_engine(connection_string)
connection = engine.connect()

In [5]:
#Getting the data for May and June from the database
may_data = pd.read_sql_query('SELECT * FROM sakila.rentals_may', engine)
june_data = pd.read_sql_query('SELECT * FROM sakila.rentals_june', engine)

In [12]:
#Checking the data for May
print(may_data.shape)
may_data.head(5)

(1156, 7)


,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


In [11]:
#Checking the data for June
print(june_data.shape)
june_data.head(5)

(2311, 7)


,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1158,2005-06-14 22:53:33,1632,416,2005-06-18 21:37:33,2,2006-02-15 21:30:53
1,1159,2005-06-14 22:55:13,4395,516,2005-06-17 02:11:13,1,2006-02-15 21:30:53
2,1160,2005-06-14 23:00:34,2795,239,2005-06-18 01:58:34,2,2006-02-15 21:30:53
3,1161,2005-06-14 23:07:08,1690,285,2005-06-21 17:12:08,1,2006-02-15 21:30:53
4,1162,2005-06-14 23:09:38,987,310,2005-06-23 22:00:38,1,2006-02-15 21:30:53


In [61]:
#Running the query for may
query_may = '''
    select c.customer_id, 
    concat(c.first_name, ' ', c.last_name) as customer_name, 
    count(rm.rental_id) as num_rentals_may
    from rentals_may rm
    join customer c on rm.customer_id = c.customer_id
    group by rm.customer_id;
'''

In [62]:
#Getting and storing the results of the query
result_may = connection.execute(query_may)

In [63]:
#Looping through the result and appending to a list
data_may = []
for row in result_may:
    data_may.append(row)

In [64]:
#Running the query for June
query_june = '''
    select c.customer_id, 
    concat(c.first_name, ' ', c.last_name) as customer_name, 
    count(rj.rental_id) as num_rentals_june
    from rentals_june rj
    join customer c on rj.customer_id = c.customer_id
    group by rj.customer_id;
'''

In [65]:
#Getting and storing the results of the query
result_june = connection.execute(query_june)

In [66]:
#Looping through the result and appending to a list
data_june = []
for row in result_june:
    data_june.append(row)

In [68]:
#Creating the final DataFrames
columns = ['Customer_ID','Customer_Name', 'Number_of_Rentals']
df_may = pd.DataFrame(data_may, columns=columns)
df_june = pd.DataFrame(data_june, columns=columns)

In [70]:
#Checking the dataframe
df_may.head()

,Customer_ID,Customer_Name,Number_of_Rentals
0,1,MARY SMITH,2
1,2,PATRICIA JOHNSON,1
2,3,LINDA WILLIAMS,2
3,5,ELIZABETH BROWN,3
4,6,JENNIFER DAVIS,3


In [71]:
#Checking the dataframe
df_june.head()

,Customer_ID,Customer_Name,Number_of_Rentals
0,1,MARY SMITH,7
1,2,PATRICIA JOHNSON,1
2,3,LINDA WILLIAMS,4
3,4,BARBARA JONES,6
4,5,ELIZABETH BROWN,5


In [106]:
def compare_borrowing(may_df, june_df):
    may_df = may_df.groupby(['Customer_ID', 'Customer_Name'], as_index=False)['Number_of_Rentals'].sum()
    june_df = june_df.groupby(['Customer_ID', 'Customer_Name'], as_index=False)['Number_of_Rentals'].sum()
    #Merging the dataframes
    merged_df = pd.merge(may_df, june_df, on=['Customer_ID', 'Customer_Name'], how='left', suffixes=('_may', '_june'))
    # Check if customer borrowed more films in June
    merged_df['Borrowed_More_in_June'] = merged_df['Number_of_Rentals_june'] > merged_df['Number_of_Rentals_may']

    return merged_df

In [107]:
#Checking the function
compared = compare_borrowing(df_may, df_june)
compared

,Customer_ID,Customer_Name,Number_of_Rentals_may,Number_of_Rentals_june,Borrowed_More_in_June
0,1,MARY SMITH,2,7.0,True
1,2,PATRICIA JOHNSON,1,1.0,False
2,3,LINDA WILLIAMS,2,4.0,True
3,5,ELIZABETH BROWN,3,5.0,True
4,6,JENNIFER DAVIS,3,4.0,True
...,...,...,...,...,...
515,594,EDUARDO HIATT,4,6.0,True
516,595,TERRENCE GUNDERSON,1,2.0,True
517,596,ENRIQUE FORSYTHE,6,2.0,False
518,597,FREDDIE DUGGAN,2,3.0,True
